# ESTRATÉGIA S-RANK

- Estratégia de seleção de fundos imobiliários

- Passo 1 Aplicação de Filtros:
 1. Filtro de liquidez diária de movimentação.
 2. Filtro de Tipo de fundos imobiliários.
 3. Filtro de Estabilidade.
 4. Filtro de Idade.

In [22]:
import sqlite3
import pandas as pd 
import os
import numpy as np 

# 1 . Filtro de Liquidez

- Realizamos um filtro para verificar os fundos com menos de 200 mil reais de negociação diaria

In [2]:
#Conexão com o database
conn = sqlite3.connect(r'database_fii.db')

In [3]:
#criação do cursor para vermos as tabelas do banco
cur = conn.cursor()
cur.execute("""select name from sqlite_master where type='table';""")
cur.fetchall()

[('tipo_do_fundo',), ('ipo_data_list',), ('ranking_diario',)]

In [4]:
#selecionar a tabela ranking_diário para podermos aplicar a estratégia s-rank
df = pd.read_sql_query("""SELECT * FROM ranking_diario""",conn)
df.head()

,codigo_do_fundo,setor,preco_atual,liquidez_diaria,dividendo,dividend_yield,dy_3m_acumulado,dy_6m_acumulado,DY_(12M)_Acumulado,dy_3m_media,...,vpa,p_vpa,dy_patrimonial,variacao_patrimonial,rentabilidade_patrimonial_periodo,rentabilidade_patrimonial_acumulada,vacancia_fisica,vacancia_financeira,quantidade_ativos,hoje
0,FIVN11,Shoppings,4.36,800,0.00,0.00,0.00,0.00,0.00,0.00,...,7.13,0.61,,,,,56.00,,1,2021-02-23 09:41:27.962330
1,BZLI11,Títulos e Val Mob,15.75,423,0.00,0.00,0.00,0.00,0.00,0.00,...,11.56,1.36,,,,,,,0,2021-02-23 09:41:27.962330
2,XTED11,Lajes Corporativas,7.09,1929,0.00,0.00,0.00,0.00,0.00,0.00,...,14.99,0.47,,,,,100.00,100.00,1,2021-02-23 09:41:27.962330
3,ALMI11,Lajes Corporativas,1360.00,71,0.00,0.00,0.00,0.00,0.00,0.00,...,208.31,0.68,,,,,68.15,,1,2021-02-23 09:41:27.962330
4,DOMC11,Lajes Corporativas,439.00,42,0.00,0.00,0.00,0.00,0.00,0.00,...,1109.62,0.40,,,,,23.44,,1,2021-02-23 09:41:27.962330


In [5]:
#ver os tipos das colunas do df
df.dtypes

codigo_do_fundo                        object
setor                                  object
preco_atual                            object
liquidez_diaria                        object
dividendo                              object
dividend_yield                         object
dy_3m_acumulado                        object
dy_6m_acumulado                        object
DY_(12M)_Acumulado                     object
dy_3m_media                            object
dy_6m_media                            object
dy_12M_media                           object
dy_ano                                 object
variacao_preco                         object
rentabilidade_periodo                  object
rentabilidade_acumulada                object
patrimonio_liq                         object
vpa                                    object
p_vpa                                  object
dy_patrimonial                         object
variacao_patrimonial                   object
rentabilidade_patrimonial_periodo 

In [6]:
#criar função para converter as colunas em tipos númericos
def coerce_columns_to_numeric(df,column_list):
    df[column_list] = df[column_list].apply(pd.to_numeric, errors='coerce')

In [7]:
#converter a coluna Liquidez Diária para numeric
coerce_columns_to_numeric(df, 'liquidez_diaria')
coerce_columns_to_numeric(df, 'dividend_yield')
coerce_columns_to_numeric(df, 'dy_12M_media')

In [8]:
#removemos os FIIS com menos de 200 mil reais em liquidez diária dos ultimos doze meses
rank = df.loc[df['liquidez_diaria'] >= 20000]

# 2. Filtro de Tipos de Fundos

- Remove os fundos de incorporação ou fundos de desenvolvimento

In [9]:
type_funds = pd.read_sql_query('SELECT * FROM tipo_do_fundo',conn)

In [10]:
type_funds.columns

Index(['Código do fundo', 'Público Alvo', 'Tipo de FII'], dtype='object')

In [11]:
type_funds = type_funds.rename(columns={'Código do fundo':'codigo_do_fundo'})


In [12]:
type_funds.drop(type_funds[type_funds['Tipo de FII'] == 'Tijolo: Desenvolvimento'].index, inplace=True)

In [26]:
rank_2 = pd.merge(rank,type_funds, on='codigo_do_fundo')

In [33]:
rank_2.columns

Index(['codigo_do_fundo', 'setor', 'preco_atual', 'liquidez_diaria',
       'dividendo', 'dividend_yield', 'dy_3m_acumulado', 'dy_6m_acumulado',
       'DY_(12M)_Acumulado', 'dy_3m_media', 'dy_6m_media', 'dy_12M_media',
       'dy_ano', 'variacao_preco', 'rentabilidade_periodo',
       'rentabilidade_acumulada', 'patrimonio_liq', 'vpa', 'p_vpa',
       'dy_patrimonial', 'variacao_patrimonial',
       'rentabilidade_patrimonial_periodo',
       'rentabilidade_patrimonial_acumulada', 'vacancia_fisica',
       'vacancia_financeira', 'quantidade_ativos', 'hoje', 'Público Alvo',
       'Tipo de FII'],
      dtype='object')

# 3. Criar um filtro de estabilidade
- o filtro de estabilidade é a diferença entre a média e a mediana do DY (dividend Yield)

In [41]:
#pegar a mediana do Dividend Yield anual
rank_2['dy_mediana'] = abs(rank_2['dy_12M_media']-rank_2['dy_12M_media'].median()) 

In [46]:
rank_2['filtro estabilidade'] = rank_2['dy_12M_media'].apply(lambda x: 'Instavel' if x < 0
else 'Estavel' if x > 0
else '')

In [47]:
rank_2.head()

,codigo_do_fundo,setor,preco_atual,liquidez_diaria,dividendo,dividend_yield,dy_3m_acumulado,dy_6m_acumulado,DY_(12M)_Acumulado,dy_3m_media,...,vacancia_fisica,vacancia_financeira,quantidade_ativos,hoje,Público Alvo,Tipo de FII,DY mediana,Dif_entre_med_e_media,dy_mediana,filtro estabilidade
0,CARE11,Híbrido,0.66,80476.0,0.00,0.24,0.69,0.69,0.69,0.23,...,,,0,2021-02-23 09:41:27.962330,Geral,Tijolo: Outros,0.46,0.06,0.46,Estavel
1,CARE11,Híbrido,0.66,80476.0,0.00,0.24,0.69,0.69,0.69,0.23,...,,,0,2021-02-23 09:41:27.962330,None,None,0.46,0.06,0.46,Estavel
2,GRLV11,Logística,145.50,54618.0,0.17,0.00,0.00,0.00,0.00,0.00,...,3.70,3.40,0,2021-02-23 09:41:27.962330,Geral,Tijolo: Galpões,0.52,0.00,0.52,
3,GRLV11,Logística,145.50,54618.0,0.17,0.00,0.00,0.00,0.00,0.00,...,3.70,3.40,0,2021-02-23 09:41:27.962330,None,None,0.52,0.00,0.52,
4,HGRU11,Híbrido,129.18,66499.0,0.68,0.53,1.62,3.21,6.60,0.54,...,0.00,0.00,16,2021-02-23 09:41:27.962330,Geral,Tijolo: Educacional,0.03,0.55,0.03,Estavel


# 4. Filtro de Idade
<l> Retira fundos que tem menos de 1 ano de existência. </l>


In [48]:
ipo_list = pd.read_sql_query('SELECT * FROM ipo_data_list',conn)

In [49]:
ipo_list.columns

Index(['Nome ', 'Código do fundo', 'Data_Ipo', 'Valor Ipo', 'Segmento',
       'Administrador '],
      dtype='object')

<l> 4.1 Renomeamos as colunas do código do Fundo </l>


In [52]:
ipo_list = ipo_list.rename(columns={'Código do fundo':'codigo_do_fundo'})

In [85]:
nr = pd.merge(rank_2,ipo_list, on='codigo_do_fundo')

In [86]:
nr.columns

Index(['codigo_do_fundo', 'setor', 'preco_atual', 'liquidez_diaria',
       'dividendo', 'dividend_yield', 'dy_3m_acumulado', 'dy_6m_acumulado',
       'DY_(12M)_Acumulado', 'dy_3m_media', 'dy_6m_media', 'dy_12M_media',
       'dy_ano', 'variacao_preco', 'rentabilidade_periodo',
       'rentabilidade_acumulada', 'patrimonio_liq', 'vpa', 'p_vpa',
       'dy_patrimonial', 'variacao_patrimonial',
       'rentabilidade_patrimonial_periodo',
       'rentabilidade_patrimonial_acumulada', 'vacancia_fisica',
       'vacancia_financeira', 'quantidade_ativos', 'hoje', 'Público Alvo',
       'Tipo de FII', 'DY mediana', 'Dif_entre_med_e_media', 'dy_mediana',
       'filtro estabilidade', 'Nome ', 'Data_Ipo', 'Valor Ipo', 'Segmento',
       'Administrador '],
      dtype='object')

In [87]:
import datetime

In [88]:
today = datetime.date.today()

In [89]:
d = today.day
m = today.month
y = today.year

In [90]:
l_y = y-1

In [91]:
data = '{}/0{}/{}'.format(d,m,l_y)

In [92]:
data

'1/03/2020'

In [101]:
nra = nr.loc[(nr['Data_Ipo'] > data) & (nr['Data_Ipo'] < data)]

In [102]:
nr.drop_duplicates('codigo_do_fundo',inplace=True)

In [103]:
nr.columns

Index(['codigo_do_fundo', 'setor', 'preco_atual', 'liquidez_diaria',
       'dividendo', 'dividend_yield', 'dy_3m_acumulado', 'dy_6m_acumulado',
       'DY_(12M)_Acumulado', 'dy_3m_media', 'dy_6m_media', 'dy_12M_media',
       'dy_ano', 'variacao_preco', 'rentabilidade_periodo',
       'rentabilidade_acumulada', 'patrimonio_liq', 'vpa', 'p_vpa',
       'dy_patrimonial', 'variacao_patrimonial',
       'rentabilidade_patrimonial_periodo',
       'rentabilidade_patrimonial_acumulada', 'vacancia_fisica',
       'vacancia_financeira', 'quantidade_ativos', 'hoje', 'Público Alvo',
       'Tipo de FII', 'DY mediana', 'Dif_entre_med_e_media', 'dy_mediana',
       'filtro estabilidade', 'Nome ', 'Data_Ipo', 'Valor Ipo', 'Segmento',
       'Administrador '],
      dtype='object')

In [104]:
nr['data_extracao'] = today

In [106]:
nr.to_sql('todos_os_fundos', schema=None, if_exists='append', index=False, con=conn)

In [107]:
nr.to_excel(r's_rank{}.xlsx'.format(str(today)),index=False)